# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [101]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [102]:
#Calculating the no. of total schools
total_school = school_data_complete["school_name"].nunique()

#Calculating the no. of total students
total_students = school_data_complete["Student ID"].count()

#Calculating total school budget
total_budget = school_data_complete["budget"].unique().sum()

#Calculating average maths score
avg_math = school_data_complete["math_score"].mean()

#Calculating average reading score
avg_reading = school_data_complete["reading_score"].mean()


#Calculating % students who passed maths (maths score >= 70)
passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70] ["math_score"].count()
passing_math_total = passing_math/total_students

#Calculating % students who passed reading (reading score >= 70)
passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70] ["reading_score"].count()
passing_reading_total = passing_reading/total_students

#Calculating Overall Passing rate
overall_pass = (passing_math_total+passing_reading_total)/2


#Creating Data frame using above results and storing it in another data frame
summary_df = pd.DataFrame({
    
    "Total Schools": [total_school],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math],
    "Average Reading Score": [avg_reading],
    "% Passing Math": [passing_math_total],
    "% Passing Reading":[passing_reading_total],
    "% Overall Passing Rate": [overall_pass]

})
summary_new = summary_df


#Formatting the data using style.format
summary_new.style.format({"Total Students": "{:,}","Total Budget": "${:,.2f}","Average Math Score": "{:.2f}",
                          "Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}",
                          "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [103]:
#Total Students by School
student_school = school_data_complete.groupby('school_name').count()[["Student ID"]]


#Get budget, type and school name. Setting school_name as index
school_type_budget = school_data_complete[['school_name', 'type','budget']].drop_duplicates().set_index('school_name')


#Calculating average maths and reading scores
avg_score = school_data_complete.groupby('school_name').mean()[['math_score', 'reading_score']]


#Get students who passed maths test
maths_pass = pd.DataFrame(school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name").count()['math_score'])


#Get students who passed reading test
reading_pass = pd.DataFrame(school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name").count()['reading_score'])


#Calculating overall passing rate by taking average of maths & reading pass students
overall_pass = (maths_pass+reading_pass)/2


#merging student_school and avg_score to get Total Students & Math and Reading average Score
summary_sch = pd.merge(student_school, avg_score, on='school_name')

#merging above summary_sch with School_typ_budget to get the school type & budget
summary_sch = pd.merge(summary_sch, school_type_budget, on='school_name')

#merging above summary_sch with maths_pass to get the students who passed maths
summary_sch = pd.merge(summary_sch, maths_pass, on='school_name')

#merging above summary_sch with reading_pass to get the students who passed reading
summary_sch = pd.merge(summary_sch, reading_pass, on='school_name')

#summary_sch

#Inserting % Passing Math snd calculating % of students who passed maths
summary_sch['% Passing Math']= (summary_sch['math_score_y']/summary_sch['Student ID'])

#Inserting % Passing Reading snd calculating % of students who passed reading
summary_sch['% Passing Reading']= (summary_sch['reading_score_y']/summary_sch['Student ID'])

#Inserting % Overall Passing Rate snd calculating by taking average of maths and reading passing
summary_sch['% Overall Passing Rate']= (summary_sch['% Passing Math']+summary_sch['% Passing Reading'])/2

#Inserting Per School Budget by dividing budget and size
summary_sch['Per Student Budget'] = summary_sch['budget']/summary_sch['Student ID']

#Renaming some of the solumns in the data frame
summary_sch = summary_sch.rename(columns={'type': 'School Type', 'Student ID': 'Total Students', 'budget': 'Total School Budget',
                                               'math_score_x': 'Average Math Score',
                                               'reading_score_x': 'Average Reading Score'})

#Getting only required columns in new data frame
summary_sch_clean = summary_sch[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing Rate']]

#Doing some formatting to include ',', '%', '.' using style.format()
summary_new = summary_sch_clean.style.format({"Total Students": "{:,}","Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}",
                                "Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})
summary_new
 



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [107]:
#Getting the top 5 performing schools using sort_values()

top_perf_schools = summary_sch_clean.sort_values('% Overall Passing Rate', ascending = False)
top_perf_schools.index.name = "School Name"
top_perf_schools.head().style.format({"Total Students": "{:,}","Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}",
                                "Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [108]:
#Getting the bottom 5 performing schools using sort_values()

bottom_perf_schools = summary_sch_clean.sort_values('% Overall Passing Rate', ascending = True)
bottom_perf_schools.index.name = "School Name"
bottom_perf_schools.head().style.format({"Total Students": "{:,}","Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}",
                                "Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [66]:
#Getting the maths scores by each grade for all schools
maths_9th = school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean()
maths_10th = school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean()
maths_11th = school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean()
maths_12th = school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()

#Creating Data Frame for above values
all_grades_maths = pd.DataFrame({'9th': maths_9th, '10th': maths_10th, '11th': maths_11th, '12th': maths_12th})

#Formatting to show 2 decimal place value
all_grades_maths.style.format({'9th': '{:.2f}', '10th': '{:.2f}', '11th': '{:.2f}', '12th': '{:.2f}'})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [67]:
#Getting the reading scores by each grade for all schools
reading_9th = school_data_complete.loc[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
reading_10th = school_data_complete.loc[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
reading_11th = school_data_complete.loc[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
reading_12th = school_data_complete.loc[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

#Creating Data Frame for above values
all_grades_reading = pd.DataFrame({'9th': reading_9th, '10th': reading_10th, '11th': reading_11th, '12th': reading_12th})

#Formatting to show 2 decimal place value
all_grades_reading.style.format({'9th': '{:.2f}', '10th': '{:.2f}', '11th': '{:.2f}', '12th': '{:.2f}'})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [69]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 800000]
group_names = ["<$585", "$585-615", "$615-645", ">$645"]
spending_bins

[0, 585, 615, 645, 800000]

In [84]:
#slicing the bins and inserting it as a new column in the data frame
school_data_complete['score_spending'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], spending_bins, 
                                                labels=group_names)

#Creating a group on the new column score_spending
spending_grp = school_data_complete.groupby('score_spending')

#calculating average maths score
math_score_avg = spending_grp['math_score'].mean()

#calculating average reading score
reading_score_avg = spending_grp['reading_score'].mean()

#Calculating the maths passing rate
maths_pass_spending = (school_data_complete[school_data_complete["math_score"] >= 70].groupby("score_spending").count()['math_score'])/spending_grp['Student ID'].count()

#Calculating the reading passing rate
reading_pass_spending = (school_data_complete[school_data_complete["reading_score"] >= 70].groupby("score_spending").count()['reading_score'])/spending_grp['Student ID'].count()

#Calculating the Overall passing rate
overall_pass_spending = (maths_pass_spending+reading_pass_spending)/2

#Creating the data frame based on the above values
spending_score_df = pd.DataFrame({"Average Math Score": math_score_avg, "Average Reading Score": reading_score_avg,
                                 "% Passing Math": maths_pass_spending, "% Passing Reading": reading_pass_spending,
                                 "% Overall Passing Rate": overall_pass_spending})

#Assigning new name to index
spending_score_df.index.name = "Spending Ranges (Per Student)"

#Formatting to include the % and decimal points
spending_score_df.style.format({"Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})





,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70%,96.69%,95.19%
$585-615,83.53,83.84,94.12%,95.89%,95.01%
$615-645,78.06,81.43,71.40%,83.61%,77.51%
>$645,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [85]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (>2000)"]

In [109]:
#slicing the bins and inserting it as a new column in the data frame
school_data_complete['score_size'] = pd.cut(school_data_complete['size'], size_bins, labels=group_names)

#Creating a group on the new column score_size
size_grp = school_data_complete.groupby('score_size')

#calculating average maths score
math_score_avg = size_grp['math_score'].mean()

#calculating average reading score
reading_score_avg = size_grp['reading_score'].mean()

#Calculating the maths passing rate
maths_pass = (school_data_complete[school_data_complete["math_score"] >= 70].groupby("score_size").count()['math_score'])/spending_grp['Student ID'].count()

#Calculating the reading passing rate
reading_pass = (school_data_complete[school_data_complete["reading_score"] >= 70].groupby("score_size").count()['reading_score'])/spending_grp['Student ID'].count()

#Calculating the overall passing rate
overall_pass = (maths_pass+reading_pass)/2


#Creating the data frame based on the above values
size_score_df = pd.DataFrame({"Average Math Score": math_score_avg, "Average Reading Score": reading_score_avg,
                                 "% Passing Math": maths_pass, "% Passing Reading": reading_pass,
                                 "% Overall Passing Rate": overall_pass})

#Assigning new name to index
size_score_df.index.name = "School Size"

#Formatting to include the % and decimal points
size_score_df.style.format({"Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,95.00%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,95.19%
Large (>2000),77.48,81.20,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [97]:
#Creating group on type
schooltype_grp = school_data_complete.groupby('type')

#Calculating the math average score
math_score_avg = schooltype_grp['math_score'].mean()

#Calculating the reading average score
reading_score_avg = schooltype_grp['reading_score'].mean()

#Calculating the math passing rate
maths_pass = (school_data_complete[school_data_complete["math_score"] >= 70].groupby("type").count()['math_score'])/schooltype_grp['Student ID'].count()

#Calculating the reading passing rate
reading_pass = (school_data_complete[school_data_complete["reading_score"] >= 70].groupby("type").count()['reading_score'])/schooltype_grp['Student ID'].count()

#Calculating the overall passing rate
overall_pass = (maths_pass+reading_pass)/2


#Creating the data frame based on the above values
type_score_df = pd.DataFrame({"Average Math Score": math_score_avg, "Average Reading Score": reading_score_avg,
                                 "% Passing Math": maths_pass, "% Passing Reading": reading_pass,
                                 "% Overall Passing Rate": overall_pass})

#Assigning new name to index
type_score_df.index.name = "School Type"

#Formatting to include the % and decimal points
type_score_df.style.format({"Average Math Score": "{:.2f}","Average Reading Score": "{:.2f}","% Passing Math": "{:.2%}", 
                                "% Passing Reading":"{:.2%}","% Overall Passing Rate": "{:.2%}"})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
